In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('bmh')

import tensorflow as tf
from tensorflow import keras

In [2]:
# Loading Data
data_train = np.load('data_train.npy')
labels_train = np.load('labels_train_corrected.npy')

print(data_train.shape, labels_train.shape)

(270000, 5933) (5933,)


In [3]:
# Labels Encoding

labels_names = ['Nike',
                'Adidas',
                'Ford',
                'Honda',
                'General Mills',
                'Unilever',
                "McDonald's",
                'KFC',
                'Gators',
                '3M']

labels_names = ['Nike',
                'Adidas',
                'Ford',
                'Honda',
                'General_mills',
                'Unilever',
                "Mcdonalds",
                'KFC',
                'Gators',
                '3M']

In [4]:
#Resizing images

from PIL import Image
import numpy as np

image_resized = []
dim = (64, 64)

for i in range(len(data_train.T)):
    image = data_train[:, i].reshape((300, 300, 3))

    # Convert RGB to BGR
    bgr_image = image[..., ::-1]

    image_pil = Image.fromarray(bgr_image.astype('uint8'))  # Convert to PIL image
    resized_pil = image_pil.resize(dim, Image.ANTIALIAS)  # Resize using Pillow's resize function

    # Convert back to BGR
    resized_bgr = np.array(resized_pil)
    resized_rgb = resized_bgr[..., ::-1]

    image_resized.append(resized_rgb)

# Convert the list of resized images to a NumPy array
image_resized = np.array(image_resized)


C:\Users\renii\AppData\Local\Temp\ipykernel_17736\3583885791.py:16: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_pil = image_pil.resize(dim, Image.ANTIALIAS)  # Resize using Pillow's resize function


In [5]:
X_train_full = np.load('data_train.npy').T
t_train_full = np.load('labels_train_corrected.npy')

X_train_full.shape, t_train_full.shape# Loading Data
data_train = np.load('data_train.npy')
labels_train = np.load('labels_train_corrected.npy')

print(data_train.shape, labels_train.shape)

(270000, 5933) (5933,)


In [6]:
from sklearn.model_selection import train_test_split

# Training and Test sets
X_training, X_test, t_training, t_test = train_test_split(X_train_full, 
                                                  t_train_full, 
                                                  shuffle=True,
                                                  stratify=t_train_full,
                                                  test_size=0.15)
# Train and validation sets
X_train, X_val, t_train, t_val = train_test_split(X_training, 
                                                  t_training, 
                                                  shuffle=True,
                                                  stratify=t_training,
                                                  test_size=0.2)

X_training.shape, t_training.shape, X_train.shape, t_train.shape, X_val.shape, t_val.shape

((5043, 270000), (5043,), (4034, 270000), (4034,), (1009, 270000), (1009,))

In [7]:
X_training_reshaped = X_training.reshape(-1, 300, 300, 3)
X_test_reshaped = X_test.reshape(-1, 300, 300, 3)

# Reshape the input data to match the model's expected input shape
X_train_reshaped = X_train.reshape(-1, 300, 300, 3)
X_val_reshaped = X_val.reshape(-1, 300, 300, 3)

In [8]:
base_model = keras.applications.resnet50.ResNet50(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.

In [9]:
base_model.add()

AttributeError: 'Functional' object has no attribute 'add'

In [10]:
base_model = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.

83683744/83683744 [==============================] - 2s 0us/step


In [11]:
# Freeze base model
base_model.trainable = False

In [12]:
IMG_SIZE = 150

# .Input() instantiates a Keras tensor
inputs = keras.Input(shape=(300, 300, 3))
# Input layer

inputs_resized = tf.keras.layers.Resizing(IMG_SIZE, IMG_SIZE)(inputs)
# resizing input to match pretrained model

x = base_model(inputs_resized, training=False)
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.

In [13]:
inputs.shape, inputs_resized.shape, x.shape

(TensorShape([None, 300, 300, 3]),
 TensorShape([None, 150, 150, 3]),
 TensorShape([None, 5, 5, 2048]))

In [14]:
# Option 2: Flattening

# Convert features of shape `base_model.output_shape[1:]` to vectors
x_flatten = keras.layers.Flatten()(x)

x_flatten.shape

TensorShape([None, 51200])

In [15]:
# A Dense classifier with 10 units and softmax activation function
outputs = keras.layers.Dense(10, activation='softmax')(x_flatten)

model = keras.Model(inputs, outputs)

In [16]:
type(model)

keras.engine.functional.Functional

In [17]:
inputs.shape, inputs_resized.shape, x.shape, outputs.shape

(TensorShape([None, 300, 300, 3]),
 TensorShape([None, 150, 150, 3]),
 TensorShape([None, 5, 5, 2048]),
 TensorShape([None, 10]))

In [18]:
model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=keras.optimizers.Nadam(learning_rate=0.01, beta_1=0.9, beta_2=0.999),
              metrics=['accuracy'])

model.fit(X_train_reshaped,t_train, epochs=5, batch_size=32,
          validation_data=(X_val_reshaped, t_val),
          callbacks=[tf.keras.callbacks.EarlyStopping(patience=2)])

# Again, in practice, you would run for a lot more epochs. 
# As well as perform the necessary hyperparameter tuning.

Epoch 1/5
127/127 [==============================] - 120s 917ms/step - loss: 628.5936 - accuracy: 0.3267 - val_loss: 1041.0428 - val_accuracy: 0.2458
Epoch 2/5
127/127 [==============================] - 110s 866ms/step - loss: 373.2649 - accuracy: 0.5151 - val_loss: 373.8035 - val_accuracy: 0.5025
Epoch 3/5
127/127 [==============================] - 113s 892ms/step - loss: 280.9030 - accuracy: 0.6113 - val_loss: 637.2619 - val_accuracy: 0.3479
Epoch 4/5
127/127 [==============================] - 110s 870ms/step - loss: 249.8238 - accuracy: 0.6492 - val_loss: 478.9712 - val_accuracy: 0.4817


In [19]:
# label predictions
y_test = np.argmax(model.predict(X_test_reshaped),axis=1)

y_test.shape

28/28 [==============================] - 20s 684ms/step


(890,)

In [20]:
from sklearn.metrics import classification_report

print(classification_report(t_test, y_test, target_names=labels_names))

               precision    recall  f1-score   support

         Nike       0.64      0.32      0.43        90
       Adidas       0.57      0.44      0.50        88
         Ford       0.65      0.57      0.61        88
        Honda       0.77      0.38      0.50        88
General_mills       0.64      0.59      0.61        90
     Unilever       0.55      0.60      0.58        91
    Mcdonalds       0.49      0.40      0.44        88
          KFC       0.90      0.10      0.18        88
       Gators       0.45      0.65      0.53        88
           3M       0.28      0.82      0.42        91

     accuracy                           0.49       890
    macro avg       0.59      0.49      0.48       890
 weighted avg       0.59      0.49      0.48       890

